In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Define values for rate constants (k)
k1 = 100000000     # rate constant for E + S --> ES
k2 = 100           # rate constant for ES --> E + S
k3 = 100000000     # rate constant for E + I --> EI (non-covalent)
k4 = 10            # rate constant for EI (non-covalent) --> E + I
k5 = 0.001         # rate constant for EI (non-covalent) --> EI* (covalent)
k6 = 0             # rate constant for EI* (covalent) --> EI (non-covalent)
kcat = 1           # rate constant for ES --> E + P
kEdeg = 0.00003    # rate constant for E --> Edeg
kESdeg = 0.00003   # rate constant for ES --> ESdeg
kEIdeg = 0.00003   # rate constant for EI --> EIdeg
kEIideg = 0.00003  # rate constant for EI* --> EI*deg


# Define concentrations at start of measurement t=0 (in V0, before addition of substrate)
# (These values are not assumed to be constant during the simulation)

E0 = 1*10**-12  # Unbound enzyme concentration E before dilution! (In V0)
ES0 = 0         # Non-covalent enzyme-substrate complex ES (in V0)
P0 = 0          # Product (in V0)
EI0 = 0         # Non-covalent enzyme-inhibitor complex EI (in V0)
EIi0 = 0        # Covalent enzyme-inhibitor complex (EI*) (in V0)
Edeg = 0        # Degraded E (in V0)
ESdeg = 0       # Degraded ES (in V0)
EIdeg = 0       # Degraded EI (in V0)
EIideg = 0      # Degraded EI* (in V0)



# Define assay constants 
# (These values are assumed to be constant during the simulation, but are updated once when the volume changes)
V0 = 1       # Volume before addition of the substrate
VS = 99      # Volume of the added substrate, assumed is that S0 = S/VS (so concentration in the volume of just substrate, not the final volume)
Vtot = V0+VS # Total volume after addition of substrate


Is = [1*10**-6, 0.5*10**-6, 0.25*10**-6, 0.15*10**-6, 0.075*10**-6, 0.04*10**-6, 0.02*10**-6, 10*10**-9, 0*10**-9] # List of inhibitor concentrations (in V0). Concentration before dilution! Can only be assumed constant if inhibitor concentration is >10*E0.
S0 = 100*10**-9 # Substrate concentration before mixing (in VS)! This is assumed to be constant after addition, which is only true if substrate conversion is less than 10%
dt = 0.001   # Seconds per simulation step (dt). Maximum value for dt defined by k1*S0*dt<1 and k3*I0*dt <1

tframe = 60   # Save a frame every 'tframe' seconds
tpincub = 600 # Preincubate with inhibitor for 'tpincub' seconds
tsub = 600    # Run with substrate for 'tsub' seconds

csv_file = "KinDilDeg.csv" # filename of the .csv output file

In [ ]:
all_conc = []
all_t0 = []
all_t1 = []
for I in Is: # Loop over all I's
    all_conc.append(np.array([E0, ES0, P0, EI0, EIi0, Edeg, ESdeg, EIdeg, EIideg], dtype=np.double)) # Matrix of concentrations
    # Before addition of of S

    S = 0  # No S in the system
        
                          # [               E,              ES, P,            EI,         EI*, Edeg, ESdeg, EIdeg, EIideg]
    all_t0.append(np.array([[-k3*I-k1*S-kEdeg,         k2+kcat, 0,            k4,           0,    0,     0,     0,      0],  #E
                            [            k1*S, -k2-kcat-kESdeg, 0,             0,           0,    0,     0,     0,      0],  #ES
                            [               0,            kcat, 0,             0,           0,    0,     0,     0,      0],  #P
                            [            k3*I,               0, 0, -k4-k5-kEIdeg,          k6,    0,     0,     0,      0],  #EI
                            [               0,               0, 0,            k5, -k6-kEIideg,    0,     0,     0,      0],  #EI*
                            [           kEdeg,               0, 0,             0,           0,    0,     0,     0,      0],  #Edeg
                            [               0,          kESdeg, 0,             0,           0,    0,     0,     0,      0],  #ESdeg
                            [               0,               0, 0,        kEIdeg,           0,    0,     0,     0,      0],  #EIdeg
                            [               0,               0, 0,             0,     kEIideg,    0,     0,     0,      0]   #EIideg
                            ]).T)

    # Change in E      = [E]*(-k3*I-k1*S-kEdeg) + [ES]*(k2+kcat)         + [P]*(0) + [EI]*(k4)            + [EI*]*(0)           + [Edeg]*(0) + [ESdeg]*(0)
    # Change in ES     = [E]*(k1*S)             + [ES]*(-k2-kcat-kESdeg) + [P]*(0) + [EI]*(0)             + [EI*]*(0)           + [Edeg]*(0) + [ESdeg]*(0)
    # Change in P      = [E]*(0)                + [ES]*(kcat)            + [P]*(0) + [EI]*(0)             + [EI*]*(0)           + [Edeg]*(0) + [ESdeg]*(0)
    # Change in EI     = [E]*(k3*I)             + [ES]*(0)               + [P]*(0) + [EI]*(-k4-k5-kEIdeg) + [EI*]*(k6)          + [Edeg]*(0) + [ESdeg]*(0)
    # Change in EI*    = [E]*(0)                + [ES]*(0)               + [P]*(0) + [EI]*(k5)            + [EI*]*(-k6-kEIideg) + [Edeg]*(0) + [ESdeg]*(0)
    # Change in Edeg   = [E]*(kEdeg)            + [ES]*(0)               + [P]*(0) + [EI]*(0)             + [EI*]*(0)           + [Edeg]*(0) + [ESdeg]*(0)
    # Change in ESdeg  = [E]*(0)                + [ES]*(kESdeg)          + [P]*(0) + [EI]*(0)             + [EI*]*(0)           + [Edeg]*(0) + [ESdeg]*(0)
    # Change in EIdeg  = [E]*(0)                + [ES]*(0)               + [P]*(0) + [EI]*(kEIdeg)        + [EI*]*(0)           + [Edeg]*(0) + [ESdeg]*(0)
    # Change in EIideg = [E]*(0)                + [ES]*(kESdeg)          + [P]*(0) + [EI]*(0)             + [EI*]*(kEIideg)     + [Edeg]*(0) + [ESdeg]*(0)



    # After addition of S
    S= S0*(VS/Vtot) # Substrate concentration after addition (assumed to be constant after this)
    I = I*(V0/Vtot) # New inhibitor concentration after S addition (assumed to be constant after dilution)
    all_t1.append(np.array([[-k3*I-k1*S-kEdeg, k2+kcat, 0, k4, 0, 0, 0, 0, 0],
                            [k1*S, -k2-kcat-kESdeg, 0, 0, 0, 0, 0, 0, 0],
                            [0, kcat, 0, 0, 0, 0, 0, 0, 0],
                            [k3*I, 0, 0, -k4-k5-kEIdeg, k6, 0, 0, 0, 0],
                            [0, 0, 0, k5, -k6-kEIideg, 0, 0, 0, 0],
                            [kEdeg, 0, 0, 0, 0, 0, 0, 0, 0],
                            [0, kESdeg, 0, 0, 0, 0, 0, 0, 0],
                            [0, 0, 0, kEIdeg, 0, 0, 0, 0, 0],
                            [0, 0, 0, 0, kEIideg, 0, 0, 0, 0]]).T)
    
# Make a list of copies of the start concentrations for each inhibitorconcentration
concentrations = np.stack(all_conc)  
t0 = np.stack(all_t0)
t1 = np.stack(all_t1)                
plot = [concentrations.copy()] # Make a list for plotting

timeconc = concentrations.copy()
t = 0
# Before addition of S (tpincub/dt steps)
for i in range(int(tpincub/dt)):
    t += 1
    timeconc += np.dot(timeconc, t0*dt).diagonal().T
    
    if (timeconc < 0).any():
        raise ValueError("Got a negative concentration, try decreasing dt")

    if t % int(tframe/dt)==0:
        print(f"frame: {int(t / (tframe//dt))}")
        plot.append(timeconc.copy()) # Add the current concentrations to the list

print("Adding substrate")
# After addition of S (tsub/dt steps)
timeconc *= (V0/Vtot) # Dilute all current concentrations
for i in range(int(tsub/dt)):
    t += 1
    timeconc += np.dot(timeconc, t1*dt).diagonal().T   
    if (timeconc < 0).any():
        raise ValueError("Got a negative concentration, try decreasing dt")

    if t % int(tframe/dt)==0:
        print(f"frame: {int(t / (tframe//dt))}")
        plot.append(timeconc.copy()) # Add the current concentrations to the list

In [ ]:
all_plots = np.array(plot).swapaxes(0,1)  # Swap data axis 0 and axis 1

In [ ]:
# This will make all the plots and dataframes
dataframes = []
fig, axes = plt.subplots(len(all_plots), 2, figsize=(12,len(all_plots)*6))
for n, plot in enumerate(all_plots):
    # Split the lists of concentrations into E, ES, P, EI en EI*, ...
    Eplot = [] 
    ESplot = []
    Pplot = []
    EIplot = []
    EIiplot = []
    EItot = []
    Edegplot = []
    ESdegplot = []
    EIdegplot = []
    EIidegplot = []

    # Loop over all the frames and add the data to the right list
    for frame in plot:
        Eplot.append(frame[0])
        ESplot.append(frame[1])
        Pplot.append(frame[2])
        EIplot.append(frame[3])
        EIiplot.append(frame[4])
        EItot.append(frame[3]+frame[4]) 
        Edegplot.append(frame[5])
        ESdegplot.append(frame[6])
        EIdegplot.append(frame[7])
        EIidegplot.append(frame[8])
    
    # Select 2 plots to start filling with data
    if len(all_plots) == 1:
        ax0 = axes[0]
        ax1 = axes[1]
    else:
        ax0 = axes[n,0]
        ax1 = axes[n,1]
    
    # Plot the figures
    labels = ('E', 'ES', "P", "EI", "EI*", "EI+Ei*", "Edeg", "ESdeg", "EIdeg", "EI*deg")
    
    for i,y in enumerate((Eplot, ESplot, Pplot, EIplot, EIiplot, EItot, Edegplot, ESdegplot, EIdegplot, EIidegplot)):
        ax0.plot(y, label=labels[i])
    ax0.legend()
    for i,y in enumerate((Eplot, ESplot, Pplot, EIplot, EIiplot, EItot, Edegplot, ESdegplot, EIdegplot, EIidegplot)):
        ax1.plot(y, label=labels[i])
    ax1.legend()
    ax1.set_ylim(0,E0)  # Zoom in to E0
    ax0.set_title("[I] = {}".format(str(Is[n]))) # Set title of the subplots
    ax1.set_title("[I] = {}".format(str(Is[n])))
    
    # Make a pandas.Dataframe for all the data
    df = pd.DataFrame(data={'E':Eplot,  "EI":EIplot, "EIi":EIiplot, "ES":ESplot, "P":Pplot,
                            "Edeg":Edegplot, "ESdeg":ESdegplot, "EIdeg":EIdegplot, "EIideg":EIidegplot}
                     )
    dataframes.append(df)

# Combine all the dataframes
df = pd.concat(dataframes, keys=[I for I in Is], axis=1) 

In [ ]:
# Print the dataframe
df

In [ ]:
# Print the csv if you want to copy it to your clipboard
print(df.to_csv())

In [ ]:
# Write out the csv file
df.to_csv(csv_file)

In [ ]:
from IPython.display import Markdown # Needed to generate download link
Markdown(f'<a href="./{csv_file}" download="{csv_file}"> Download {csv_file} </a>') # Print download link